In [1]:
import pandas as pd

import tables

times = [pd.to_timedelta(5, 'm'), pd.to_timedelta(3, 'h')]
df = pd.read_csv('../data/first_detection_times.csv')
df['time'] = pd.to_timedelta(df['time'])
df.head()

,campaign_id,fuzzer,subject,defect,time
0,10,Zest,Bcel,B0,0 days 00:42:48.816000
1,100,Zeugma-1PT,Rhino,R0,0 days 01:34:51.614000
2,100,Zeugma-1PT,Rhino,R1,0 days 00:44:59.002000
3,100,Zeugma-1PT,Rhino,R2,0 days 00:49:16.492000
4,100,Zeugma-1PT,Rhino,R3,0 days 00:00:13.996000


In [2]:
def create_detection_table(df, times):
    elements = []
    for time in times:
        temp = df.copy()
        temp['detected'] = temp['time'] < time
        temp['time'] = time
        elements.append(temp)
    return pd.concat(elements)


detecting = create_detection_table(df, times)
display(detecting)

,campaign_id,fuzzer,subject,defect,time,detected
0,10,Zest,Bcel,B0,0 days 00:05:00,False
1,100,Zeugma-1PT,Rhino,R0,0 days 00:05:00,False
2,100,Zeugma-1PT,Rhino,R1,0 days 00:05:00,False
3,100,Zeugma-1PT,Rhino,R2,0 days 00:05:00,False
4,100,Zeugma-1PT,Rhino,R3,0 days 00:05:00,True
...,...,...,...,...,...,...
1875,643,Zeugma-2PT,Nashorn,N2,0 days 03:00:00,False
1876,643,Zeugma-2PT,Nashorn,N1,0 days 03:00:00,False
1877,532,Zeugma-1PT,Nashorn,N0,0 days 03:00:00,False
1878,532,Zeugma-1PT,Nashorn,N2,0 days 03:00:00,False


In [3]:
import report_util


def select(data, time, defect):
    data = data[data['time'] == time]
    return data[data['defect'] == defect]


def create_rows(data, value, target_fuzzer='Zeugma-Linked'):
    fuzzers = sorted(data['fuzzer'].unique())
    rows = []
    values1 = data[data['fuzzer'] == target_fuzzer][value]
    significance_level = 0.05 / (len(fuzzers) * (len(fuzzers) - 1) / 2)
    for fuzzer in fuzzers:
        values2 = data[data['fuzzer'] == fuzzer][value]
        sig = 'color: red;' if report_util.fisher_exact(values1, values2) < significance_level else ''
        rows.append([fuzzer, values2.mean(), sig])
    return pd.DataFrame(rows, columns=['fuzzer', 'value', 'sig'])


def create_sig_table(data, value):
    frames = []
    for time, defect in data[['time', 'defect']].drop_duplicates().itertuples(index=False, name=None):
        selected = select(data, time, defect)
        frame = create_rows(selected, value)
        frame['time'] = time
        frame['defect'] = defect
        frames.append(frame)
    return pd.concat(frames)


table = create_sig_table(detecting, 'detected')
values = tables.pivot(table, 'defect', 'fuzzer', 'value')
sigs = tables.pivot(table, 'defect', 'fuzzer', 'sig')
styler = tables.style_table(values, precision=2, axis=0).apply(lambda _: sigs, axis=None)
display(styler)

In [4]:
print(styler.to_latex(multicol_align='c', hrules=True, multirow_align='t', convert_css=True))

\begin{tabular}{lrrrrrrrrrrrrrrrrrrrrrrrr}
\toprule
Defect & \multicolumn{2}{c}{B0} & \multicolumn{2}{c}{B1} & \multicolumn{2}{c}{C0} & \multicolumn{2}{c}{C1} & \multicolumn{2}{c}{N0} & \multicolumn{2}{c}{N1} & \multicolumn{2}{c}{N2} & \multicolumn{2}{c}{R0} & \multicolumn{2}{c}{R1} & \multicolumn{2}{c}{R2} & \multicolumn{2}{c}{R3} & \multicolumn{2}{c}{R4} \\
 & 5M & 3H & 5M & 3H & 5M & 3H & 5M & 3H & 5M & 3H & 5M & 3H & 5M & 3H & 5M & 3H & 5M & 3H & 5M & 3H & 5M & 3H & 5M & 3H \\
Fuzzer &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  \\
\midrule
BeDivFuzz-Simple & 0.00 & 0.65 & {\cellcolor{yellow}} 0.00 & 0.00 & 0.00 & \color{red} 0.05 & 0.15 & {\cellcolor{yellow}} 1.00 & 0.00 & \color{red} 0.00 & {\cellcolor{yellow}} 0.00 & 0.00 & {\cellcolor{yellow}} 0.00 & \color{red} 0.00 & {\cellcolor{yellow}} 0.00 & \color{red} 0.00 & 0.05 & 0.45 & {\cellcolor{yellow}} 0.00 & \color{red} 0.00 & {\cellcolor{yellow}} 1.00 & {\cellcolor{yellow}} 1.00 & {\cellcolor{yellow}} 0